# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a video from your Google Drive

@dinatih update base on https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/OpenPose.ipynb

## Choose a video from your Google Drive

In [ ]:
!cd openpose || true && mkdir build && cd build && cmake .. && make -j`nproc`


/bin/bash: line 0: cd: openpose: No such file or directory
CMake Error: The source directory "/content" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.


## Install OpenPose on Google Colab 

*   List item
*   List item




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
# git_repo_url = 'https://github.com/dinatih/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949

  print("install new CMake becaue of CUDA10")
  if not exists('cmake-3.13.0-Linux-x86_64.tar.gz'):
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("CMakelist.txt's caffe fix")
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`





install new CMake becaue of CUDA10
clone openpose
CMakelist.txt's caffe fix
install system dependencies
Selecting previously unselected package libgflags2.2.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0

## Process Videos from Drive





In [3]:
import os
from os.path import exists, join, basename, splitext

import re 

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

folder_path = '/content/drive/My Drive/Colab_Files/EverybodyDanceNow/dataset_raw/'
colab_openpose_video_path = '/content/drive/My Drive/Colab_Files/EverybodyDanceNow/dataset_raw/'
folder_path_json = '/content/drive/My Drive/Colab_Files/EverybodyDanceNow/dataset_raw/jason_keys/'
folder_path_image = '/content/drive/My Drive/Colab_Files/EverybodyDanceNow/dataset_raw/jason_frames/'
!mkdir '{folder_path_json}'
!mkdir '{folder_path_image}'
files = os.listdir(folder_path)
files.reverse()
for filename in files:
  if filename.endswith('.mp4') and not filename.endswith('-openpose.mp4'):
    print(filename)
    colab_video_path = folder_path + filename
    print(colab_video_path)
    colab_openpose_video_path = colab_video_path.replace('.mp4', '') + '-openpose.mp4'
    print(colab_openpose_video_path)
    if not exists(colab_openpose_video_path): 
      !ffmpeg -i '{colab_video_path}' -qscale:v 2 '{folder_path_image}/'output_%03d.jpg 
      files = os.listdir(folder_path_image)
      
      sorted_frames=[]
      for x in sorted_nicely(files):
          sorted_frames.append(x)
      '''
      count = 0
      step = 3
      
      for filename in sorted_frames:
        print(filename)
        if count% step !=0:
          !rm '{folder_path_image}''{filename}'
        if count % step == 0:
          print(count)
        count = count + 1
        #every third frame
      '''
      !cd openpose && ./build/examples/openpose/openpose.bin --hand --face --image_dir '{folder_path_image}' \
       --display 0 --render_pose  0\
       --write_json '{folder_path_json}' # -vframes 6000 
      

print('done!')


normalized.mp4
/content/drive/My Drive/Colab_Files/EverybodyDanceNow/dataset_raw/normalized.mp4
/content/drive/My Drive/Colab_Files/EverybodyDanceNow/dataset_raw/normalized-openpose.mp4
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --en

In [4]:
%cd "/content/drive/My Drive/Colab_Files/EverybodyDanceNow/"
%ls

/content/drive/My Drive/Colab_Files/EverybodyDanceNow
build/                            data_prep/    openpose/     sample_data/
checkpoints/                      dataset/      options/      test_fullts.py
cmake-3.13.0-Linux-x86_64.tar.gz  dataset_raw/  __pycache__/  train_fullts.py
data/                             models/       result/       util/


In [5]:
import glob
import os
import re
def get_trailing_number(s):
	m = re.search(r'\d+$', s)
	return int(m.group()) if m else None
  
# Function to rename multiple files 
def main(mypath, name_stub, len_num, remove): 
	count = 0 
	files = glob.glob(mypath+"*.json")
	print(files)
	for current in files:
		number_trailing = re.findall('\d+',current )
		number_trailing = [s.lstrip("0") for s in number_trailing]
		
		#print(number_trailing)
		if remove==True:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints")#+".json") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints")
		else:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints"+".json") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints"+'.json')
		count = count + 1

def main_2(mypath, name_stub, len_num, remove): 
	count = 0 
	files = glob.glob(mypath+"*.jpg")
	print(files)
	for current in files:
		number_trailing = re.findall('\d+',current )
		number_trailing = [s.lstrip("0") for s in number_trailing]
		
		if remove==True:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num))#+".jpg") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num))
		else:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+".jpg") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+".jpg") 
		count = count + 1
  
  
# Driver Code 
import sys

#print(sys.argv)
if sys.argv[2] == 'json':
	main(sys.argv[1], "output_",6) 
if sys.argv[2] == 'jpg':
	main_2(sys.argv[1], "output_",6) 


In [6]:
main("./dataset_raw/jason_keys/", "output_", 6, False)
#main("./dataset_raw/close_keys/", "output_", 6, False)
#main("./dataset_raw/far_keys/", "output_", 6, False)

main_2("./dataset_raw/jason_frames/", "output_", 6, False)
#main_2("./dataset_raw/close_frames/", "output_", 6, False)
#main_2("./dataset_raw/far_frames/", "output_", 6, False)

['./dataset_raw/jason_keys/output_001_keypoints.json', './dataset_raw/jason_keys/output_002_keypoints.json', './dataset_raw/jason_keys/output_003_keypoints.json', './dataset_raw/jason_keys/output_004_keypoints.json', './dataset_raw/jason_keys/output_005_keypoints.json', './dataset_raw/jason_keys/output_006_keypoints.json', './dataset_raw/jason_keys/output_007_keypoints.json', './dataset_raw/jason_keys/output_008_keypoints.json', './dataset_raw/jason_keys/output_009_keypoints.json', './dataset_raw/jason_keys/output_010_keypoints.json', './dataset_raw/jason_keys/output_011_keypoints.json', './dataset_raw/jason_keys/output_012_keypoints.json', './dataset_raw/jason_keys/output_013_keypoints.json', './dataset_raw/jason_keys/output_014_keypoints.json', './dataset_raw/jason_keys/output_015_keypoints.json', './dataset_raw/jason_keys/output_016_keypoints.json', './dataset_raw/jason_keys/output_017_keypoints.json', './dataset_raw/jason_keys/output_018_keypoints.json', './dataset_raw/jason_keys/o

Change the --save_dir from dataset/train to dataset/test depending on situation, this one is for train dataset

In [ ]:
%cd "/content/drive/My Drive/Colab_Files/EverybodyDanceNow/"

!python3 ./data_prep/graph_train.py \
 --keypoints_dir ./dataset_raw/jason_keys/ \
 --frames_dir ./dataset_raw/jason_frames/ \
 --save_dir ./dataset/train/ \
 --spread 0 100000 1 \
 --facetexts


Streaming output truncated to the last 5000 lines.
graphing file output_007093
2365
empty 150
2366
graphing file output_007099
2367
empty 150
2368
graphing file output_007105
2369
graphing file output_007108
2370
graphing file output_007111
2371
graphing file output_007114
2372
graphing file output_007117
2373
graphing file output_007120
2374
graphing file output_007123
2375
graphing file output_007126
2376
graphing file output_007129
2377
graphing file output_007132
2378
graphing file output_007135
2379
graphing file output_007138
2380
graphing file output_007141
2381
graphing file output_007144
2382
graphing file output_007147
2383
graphing file output_007150
2384
graphing file output_007153
2385
graphing file output_007156
2386
graphing file output_007159
2387
graphing file output_007162
2388
graphing file output_007165
2389
graphing file output_007168
2390
graphing file output_007171
2391
graphing file output_007174
2392
graphing file output_007177
2393
graphing file output_007180


This one is for Validation


In [7]:
%cd "/content/drive/My Drive/Colab_Files/EverybodyDanceNow/"

!python3 ./data_prep/graph_avesmooth.py \
--keypoints_dir ./dataset_raw/jason_keys/ \
--frames_dir ./dataset_raw/jason_frames/ \
--save_dir ./dataset/test/ \
--spread 0 10000 1 \
--facetexts


/content/drive/My Drive/Colab_Files/EverybodyDanceNow
1
----------------- Loading Frames -----------------
----------------- All Loaded -----------------
0
Plotting stick figure for last frame output_000001
1
Plotting stick figure for last frame output_000002
2
Plotting stick figure for last frame output_000003
3
Plotting stick figure for last frame output_000004
4
Plotting stick figure for last frame output_000005
5
Plotting stick figure for last frame output_000006
6
Plotting stick figure for last frame output_000007
7
Plotting stick figure for last frame output_000008
8
Plotting stick figure for last frame output_000009
9
Plotting stick figure for last frame output_000010
10
Plotting stick figure for last frame output_000011
11
Plotting stick figure for last frame output_000012
12
Plotting stick figure for last frame output_000013
13
Plotting stick figure for last frame output_000014
14
Plotting stick figure for last frame output_000015
15
Plotting stick figure for last frame output

In [8]:
drive.flush_and_unmount()

Test- Actual inferencings Use

In [ ]:
%cd "/content/drive/My Drive/Colab_Files/EverybodyDanceNow/"

!python3 ./data_prep/graph_posenorm.py \
--target_keypoints ./dataset_raw/jason_keys/ \
--source_keypoints ./dataset_raw/close_keys/  \
--source_shape 1080 1920 3 \
--target_shape 1080 1920 3 \
--source_frames ./dataset_raw/jason_frames/ \
--results ./dataset/test/ \
--target_spread 0 856 \
--source_spread 0 2436 \
--calculate_scale_translation  \
--format json \
--filestart "output_" \
--x_scaling 2 \
--y_scaling 1000
#filestart is the name of the file stub


Streaming output truncated to the last 5000 lines.
(447.0040347961079, 933.54, 962.943)
(499.1945220292386, 936.475, 974.725)
(466.8852841940942, 927.666, 957.089)
(482.71258127585605, 968.83, 980.588)
(440.4890605054795, 918.819, 939.382)
(469.6347102419071, 980.649, 992.388)
(498.0407381751014, 951.116, 977.684)
(454.94153735397697, 921.715, 945.27)
(465.4932589350784, 951.185, 962.933)
(485.89264872809093, 945.247, 974.674)
(473.7084932244724, 965.906, 974.712)
(465.1970380817143, 951.18, 974.683)
(481.28117634185526, 921.726, 951.202)
(450.03791646704616, 951.133, 954.156)
(493.5977225686926, 980.587, 986.471)
(474.016241175342, 936.485, 980.641)
(438.47498195450106, 901.119, 924.697)
(489.99039664568943, 954.121, 980.613)
(448.67006346869186, 901.114, 927.583)
(451.6341832169039, 912.925, 930.55)
(464.5482061745584, 904.081, 942.361)
(498.71827518549986, 977.623, 995.309)
(466.54513384344716, 930.594, 960.029)
(435.77298090749036, 921.754, 927.597)
(512.5935587987816, 992.38, 1015